# **Course**: Deep Learning

[<img align="right" width="400" height="100" src="https://www.tu-braunschweig.de/typo3conf/ext/tu_braunschweig/Resources/Public/Images/Logos/tu_braunschweig_logo.svg">](https://www.tu-braunschweig.de/en/)

[Mehdi Maboudi](https://www.tu-braunschweig.de/en/igp/staff/mehdi-maboudi) \([m.maboudi@tu-bs.de](m.maboudi@tu-bs.de)) and [Pedro Achanccaray](https://www.tu-braunschweig.de/en/igp/staff/pedro-diaz) (p.diaz@tu-bs.de)

[Technical University of Braunschweig](https://www.tu-braunschweig.de/en/)  
[Institute of Geodesy and Photogrammetry](https://www.tu-braunschweig.de/igp)

# **Administrative topics**

**Presentation 01: Datasets**
- Presentation: 10 min., Questions: 5 min
- Update your repositories: code and presentation (PDF and/or PPTX)

# **Assignment 03:** Vanilla CNN

Let's see each group solution.



## **One possible solution**

### In the file **`datasets.py`**

- **`train_val_test_split`** function:
```python
  # TODO: Train and test split. Use test_size here.  
  x_train, x_test, y_train, y_test = train_test_split(df["path_image"].values,
                                                      df["class_int"].values,                                                     
                                                      test_size=test_size,
                                                      stratify=df["class_int"].values,
                                                      random_state=SEED)

  val_size_relative = val_size/(1-test_size)
  # TODO: Train and validation split. Use val_size_relative here.  
  x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                    y_train,
                                                    test_size=val_size_relative,
                                                    stratify=y_train,
                                                    random_state=SEED)
```



### In the file **`data_generator.py`**

- **`__init__`** function:
```python
    # TODO: Get all input parameters
    self.path_images = path_images
    self.labels = labels
    self.batch_size = batch_size
    self.n_classes = n_classes
    self.targe_size = target_size
    self.shuffle= shuffle
```

- **`on_epoch_end`** function:
```python
    if self.shuffle:
    # TODO: Shuffle image paths and their labels together
      self.path_images, self.labels = shuffle(self.path_images, self.labels)
```

- **`__get_image`** function:
```python
    # TODO: Read the image using OpenCV as the image format is JPG.
    #       Remember to include a dimension for the batches.
    x_sample = cv2.imread(path_image)    
    x_sample = cv2.resize(x_sample, (self.targe_size,self.targe_size))
    x_sample = np.expand_dims(x_sample, axis=0)
    x_sample = x_sample.astype("float")
```

- **`__get_label`** function:
```python
    # TODO: convert the label to categorical
    y_sample = to_categorical(label, num_classes=self.n_classes)
```

- **`__getitem__`** function:
```python
    # TODO: initialize the two outputs of this function
    #       x: batch with images [batch_size, width, height, channels],
    #       y: batch with labels [batch_size, n_classes]
    x = np.zeros((current_batch_size,
                  self.targe_size,
                  self.targe_size,
                  3),
                  dtype=np.float32)
    
    y = np.zeros((current_batch_size,
                  self.n_classes),
                  dtype=np.float32)
    
    # TODO: Fill x and y with data from each image and its label
    for j, (path_image,label) in enumerate(zip(batch_images,batch_labels)):
        # Reading each image
        x_sample = self.__get_image(path_image)        
        # Get the label
        y_sample = self.__get_label(label)        

        # Do not forget to normalize the data!
        x[j,...] = x_sample/255.0
        y[j,...] = y_sample
```

### In the file **`models.py`**

- **`create_cnn`** function:
```python
    # TODO: Add different layers to build a CNN: model.add(...)
    #       For this assignment, you can use the following layers:
    #       Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten,
    #       and Dense.
    #       Feel free to create your custom CNN using any of those layers.

    model = Sequential()
    # Convolutional block 1
    model.add(Conv2D(filters[0],
                    (k,k),
                    input_shape=input_shape,
                    activation="relu"))    
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))  
    model.add(Dropout(rate=0.2))
    
    # More convolutional blocks
    for n in filters[1:]:
      model.add(Conv2D(n,
                      (k,k),
                      activation="relu"))        
      model.add(BatchNormalization())
      model.add(MaxPooling2D(2,2))    
      model.add(Dropout(rate=0.2))
    
    # Flatten and output layer
    model.add(Flatten())
    model.add(Dense(100,
                    activation="relu"))    
    # Output layer
    model.add(Dense(n_classes,
                    activation="softmax"))
```

# **Lab 04:** Vanilla **C**onvolutional **N**eural **N**etworks (CNN) for Image Classification

In this lab session, we will use the [UC Merced](http://weegee.vision.ucmerced.edu/datasets/landuse.html) dataset to cover the following topics:
- CNN for image classification
- Data generators with data augmentation

This dataset has the following classes:

<center>

|Class| Description ||| Class | Description |
|:---:|:-----------:|||:-----:|:-----------:|
|  0  | agricultural|||  10   |    harbor   |
|  1  |  airplane   |||  11   |intersection |
|  2  |baseballdiamond|||12   |mediumresidential|
|  3  |   beach     |||  13   |mobilehomepark|
|  4  |  buildings  |||  14   |  overpass   |
|  5  |  chaparral  |||  15   | parkinglot  |
|  6  |denseresidential|||16  |    river    |
|  7  |    forest   |||  17   |   runway    |
|  8  |  freeway    |||  18   |sparseresidential|
|  9  | golfcourse  |||  19   | storagetanks|
|     |             |||  20   | tenniscourt |

</center>

## **Mount Google Drive to Google Colab**

You can **skip** the following cells if you **do not want to storage** the trained model in your Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Change **this path** to the **path of a folder** in your Google Drive to storage the files associated to this session (trained model, figures for visualization, etc.)

In [ ]:
# %cd drive/MyDrive/PATH/TO/YOUR/FOLDER
%cd drive/MyDrive/Colabs/DeepLearning_course/Lab_04

/content/drive/MyDrive/Colabs/DeepLearning_course/Lab_04


## **Load packages and data**

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00


In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# Management of files
import os
from os.path import exists, join

# Tensorflow and Keras
from tensorflow.keras.callbacks import ModelCheckpoint, \
                                       EarlyStopping

# Monitor training
import wandb
from wandb.keras import WandbMetricsLogger

# Working with arrays
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# External files with functions to load the dataset,
# create a CNN model, and a data generator.
from importlib import reload
import datasets
import models
import data_generator
# Useful to reload modified external files without need
# of restarting the kernel. Just run again this cell.
reload(datasets)
reload(models)
reload(data_generator)

from datasets import *
from models import *
from data_generator import *

**Variables**

In [ ]:
PROJECT_DIR = "." # os.getcwd()
SEED = 42
BATCH_SIZE = 32
TARGET_SIZE = 256
EPOCHS = 200

### **Download the dataset**

First, we will download the data for this session from the given link.

For this, we will use **`wget`** and **`zipfile`** packages to download the zip file **`data_satellite.zip`** and to extract its content.

The function **`download_ucmerced`** is defined in the file **`datasets.py`**

_In case of error during the download, you can download the dataset directly from [here](http://weegee.vision.ucmerced.edu/datasets/UCMerced_LandUse.zip)._

In [ ]:
download_ucmerced(PROJECT_DIR)

### **Reading the dataset**

The function **`read_ucmerced`** is defined in the file **`datasets.py`**

In [ ]:
path_data = join(PROJECT_DIR, "UCMerced_LandUse", "Images")

df, n_classes = read_ucmerced(path_data=path_data,
                              SEED=SEED)
classes = np.unique(df["class_str"].values)

df

,path_image,class_str,class_int
0,./UCMerced_LandUse/Images/harbor/harbor34.tif,harbor,10
1,./UCMerced_LandUse/Images/intersection/interse...,intersection,11
2,./UCMerced_LandUse/Images/agricultural/agricul...,agricultural,0
3,./UCMerced_LandUse/Images/mobilehomepark/mobil...,mobilehomepark,13
4,./UCMerced_LandUse/Images/denseresidential/den...,denseresidential,6
...,...,...,...
2095,./UCMerced_LandUse/Images/river/river38.tif,river,16
2096,./UCMerced_LandUse/Images/harbor/harbor95.tif,harbor,10
2097,./UCMerced_LandUse/Images/intersection/interse...,intersection,11
2098,./UCMerced_LandUse/Images/mediumresidential/me...,mediumresidential,12


### **Train, Validation and Test sets**

Now, we can create **three disjoint** sets: `train`, `validation` and `test`.

Let's use the following proportions:
- `train`: 60%
- `validation`: 20%
- `test`: 20%

The function **`train_val_test_split`** is defined in the file **`datasets.py`**

In [ ]:
splits = train_val_test_split(df,
                              val_size=0.2,
                              test_size=0.2,
                              SEED=SEED)

x_train = splits["x_train"]
y_train = splits["y_train"]
x_val = splits["x_val"]
y_val = splits["y_val"]
x_test = splits["x_test"]
y_test = splits["y_test"]

### **Class distribution**


For **sanity check**, let's verify the **class distribution** of each set: `train`, `validation` and `test`.

In [ ]:
# Number of samples per class
_, counts_train = np.unique(y_train, return_counts=True)
_, counts_val = np.unique(y_val, return_counts=True)
_, counts_test = np.unique(y_test, return_counts=True)

print("Samples per class - train: {}".format(counts_train))
print("Samples per class - val: {}".format(counts_val))
print("Samples per class - test: {}".format(counts_test))

Samples per class - train: [60 60 60 60 60 60 60 60 60 60 60 60 60 60 60 60 60 60 60 60 60]
Samples per class - val: [20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20]
Samples per class - test: [20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20 20]


## **Data generator with data augmentation**

There are many options to apply transformations to each sample in a dataset an increase the number of samples and their variability.

**1. [Keras ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)**

```python
tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=0,
    brightness_range=None,    
    zoom_range=0.0,    
    fill_mode='nearest',    
    horizontal_flip=False,
    vertical_flip=False,
)
```


**2. [Albumentations](https://github.com/albumentations-team/albumentations)**

<center>

**Pixel-level transformations**

<img src="https://camo.githubusercontent.com/0a0cea2503f569ac965d9de888206b482d13ffe751c682638c730296a352e1ba/68747470733a2f2f686162726173746f726167652e6f72672f776562742f62642f6e652f72762f62646e6572763563746b75646d73617a6e687734637273646669772e6a706567" width=800>

**Semantic segmentation**

<img src="https://camo.githubusercontent.com/d0235725dc1c4a25eee2d02e674f566d13e2d02fd940f52f500f06c9ae9e0797/68747470733a2f2f686162726173746f726167652e6f72672f776562742f73752f77612f6e702f737577616e70656f36777737777077746f6274727a645f636732302e6a706567" width=800>

**Object detection**

<img src="https://camo.githubusercontent.com/d11f6965edd722d6d10c369665da745fb9d5423e596726e14ec1a0090bd4ee89/68747470733a2f2f686162726173746f726167652e6f72672f776562742f727a2f2d682f336a2f727a2d68336a616c62786963386f5f6668756378797374733474632e6a706567" width=600>

</center>

**3. [scikit-image](https://scikit-image.org/)**

**4. [Augmentor](https://augmentor.readthedocs.io/en/master/)**

**5. [Torchvision](https://pytorch.org/vision/stable/index.html)**

**6. [imgaug](https://imgaug.readthedocs.io/en/latest/)**

**7. [OpenCV](https://opencv.org/)**

### **Using albumentations**

You can check the documentation here: [Albumentations documentation](https://albumentations.ai/docs/)

Each transformation has a parameter `p`, which is the probability of applying that transform.

You can use the [Composition API](https://albumentations.ai/docs/api_reference/core/composition/) to combine transformations using the `Compose`, `OneOf`, `OneOrOther`, `SomeOf`, among others.

In [ ]:
from albumentations import Blur, \
                           Compose, \
                           HorizontalFlip, \
                           RandomBrightnessContrast, \
                           RandomRotate90, \
                           VerticalFlip

def augmentation():
    return Compose([HorizontalFlip(p=0.5),
                    VerticalFlip(p=0.5),
                    RandomRotate90(p=0.5),
                    Blur(p=0.01, blur_limit=3),
                    RandomBrightnessContrast(p=0.5),
                   ], p = 1)

**How to select which transformation we should apply?**

Let's see the implementation of a data generator with data augmentation in the file **`data_generator.py`**.

In [ ]:
data_gen_train = DataGenerator(path_images=x_train,
                               labels=y_train,
                               batch_size=BATCH_SIZE,
                               n_classes=n_classes,
                               target_size=TARGET_SIZE,
                               shuffle=True)

data_gen_train_aug = DataGenerator(path_images=x_train,
                                   labels=y_train,
                                   batch_size=BATCH_SIZE,
                                   n_classes=n_classes,
                                   target_size=TARGET_SIZE,
                                   augmentation=augmentation,
                                   shuffle=True)

data_gen_val = DataGenerator(path_images=x_val,
                             labels=y_val,
                             batch_size=BATCH_SIZE,
                             n_classes=n_classes,
                             target_size=TARGET_SIZE,
                             shuffle=False)

For sanity check, you should always verify (data type, shape and visualization) the output of your data generators.

In [ ]:
for data,data_aug in zip(data_gen_train, data_gen_train_aug):
  x = data[0]
  x_aug = data_aug[0]

  plt.figure(figsize=(20,10))
  for j, (x_i,x_aug_i) in enumerate(zip(x,x_aug)):
    img = np.concatenate((x_i,np.zeros((256,10,3)),x_aug_i), axis=1)
    img = img*255
    img = img.astype(np.uint8)

    plt.subplot(5,5,j+1)
    plt.imshow(img)
    plt.axis("off")
    plt.title("Original   |   Augmented")
    if j >= 24:
      break
  plt.show()
  break

Output hidden; open in https://colab.research.google.com to view.

## **CNN for image classification**


Let's create a CNN architecture with the following parameters:
- Input shape: $256\times 256\times 3$
- Convolutional layers (Conv):
  - Kernel size ($k$): $3\times 3$
  - Number of kernels: 16, 32, 64, 128 and 256
  - Padding: 0
  - Stride: 1
  - Activation: `ReLU`
- Convolutional blocks: `Conv + BatchNorm + MaxPool + Dropout`
- Number of convolutional blocks: 5
- Dense layers: `100, n_classes`


This is a deep network with 5 convolutional layers. **How can we define the number of convolutional layers?**

In [ ]:
filters = [16, 32, 64, 128, 256]
k = 3

model_orig = create_cnn(filters=filters,
                        k=k,
                        input_shape=(TARGET_SIZE,TARGET_SIZE,3),
                        n_classes=n_classes)
model_aug = create_cnn(filters=filters,
                       k=k,
                       input_shape=(TARGET_SIZE,TARGET_SIZE,3),
                       n_classes=n_classes)

model_orig.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 batch_normalization (Batch  (None, 254, 254, 16)      64        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 16)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 127, 127, 16)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 batch_normalization_1 (Bat  (None, 125, 125, 32)      1

## **Training without data augmentation**


In [ ]:
def get_ckpt(ckpt_name):
    return ModelCheckpoint(ckpt_name,
                           mode="max",
                           save_best_only=True,
                           monitor="val_accuracy",
                           verbose=1)

### **Callbacks**

In [ ]:
# Callbacks

# Checkpoints
cb_autosave_orig = get_ckpt("cnn_orig.h5")

# Early stopping
cb_early_stop = EarlyStopping(patience=20,
                           verbose=1,
                           mode="auto",
                           monitor="val_accuracy")

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Vanilla CNN for image classification with UCMerced",
    name="vanilla-cnn-orig-classification-ucmerced",

    # track hyperparameters and run metadata
    config={
    "architecture": "Vanilla CNN",
    "dataset": "UCMerced",
    "bs": BATCH_SIZE
    }
)

cb_wandb_orig = WandbMetricsLogger()

cb_orig = [cb_autosave_orig, cb_early_stop, cb_wandb_orig]

wandb: Currently logged in as: pedro9589 (igp-tubs). Use `wandb login --relogin` to force relogin


### **Training**

In [ ]:
history_orig = model_orig.fit(data_gen_train,
                              epochs=EPOCHS,
                              validation_data=data_gen_val,
                              callbacks=cb_orig
                              )
wandb.finish()

Epoch 1/200
40/40 [==============================] - ETA: 0s - loss: 3.4589 - accuracy: 0.1587
Epoch 1: val_accuracy improved from -inf to 0.04762, saving model to cnn_orig.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


40/40 [==============================] - 463s 11s/step - loss: 3.4589 - accuracy: 0.1587 - val_loss: 4.3183 - val_accuracy: 0.0476
Epoch 2/200
40/40 [==============================] - ETA: 0s - loss: 2.2274 - accuracy: 0.3278
Epoch 2: val_accuracy improved from 0.04762 to 0.05714, saving model to cnn_orig.h5
40/40 [==============================] - 10s 255ms/step - loss: 2.2274 - accuracy: 0.3278 - val_loss: 7.4445 - val_accuracy: 0.0571
Epoch 3/200
40/40 [==============================] - ETA: 0s - loss: 1.7880 - accuracy: 0.4762
Epoch 3: val_accuracy did not improve from 0.05714
40/40 [==============================] - 9s 233ms/step - loss: 1.7880 - accuracy: 0.4762 - val_loss: 8.4369 - val_accuracy: 0.0476
Epoch 4/200
40/40 [==============================] - ETA: 0s - loss: 1.4367 - accuracy: 0.5730
Epoch 4: val_accuracy improved from 0.05714 to 0.06190, saving model to cnn_orig.h5
40/40 [==============================] - 10s 245ms/step - loss: 1.4367 - accuracy: 0.5730 - val_loss: 

epoch/accuracy,▁▂▄▄▅▆▆▇▇███████████████████████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▁▁▁▁▁▂▂▂▃▃▃▃▅▇▇█▇█▇███▆▇▅▆▆▇▇▇██▆▇█▇██▆
epoch/val_loss,▂▄▄▅▆█▆▆▅▄▃▃▃▂▁▂▁▁▁▂▁▁▁▂▁▂▂▃▂▂▁▁▁▃▁▁▁▁▁▃
epoch/accuracy,0.97381
epoch/epoch,45
epoch/learning_rate,0.001
epoch/loss,0.09554
epoch/val_accuracy,0.5119


## **Training with data augmentation**

### **Callbacks**

In [ ]:
# Callbacks

# Checkpoints
cb_autosave_aug = get_ckpt("cnn_aug.h5")

# Early stopping
cb_early_stop = EarlyStopping(patience=20,
                           verbose=1,
                           mode="auto",
                           monitor="val_accuracy")

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Vanilla CNN for image classification with UCMerced",
    name="vanilla-cnn-aug-classification-ucmerced",

    # track hyperparameters and run metadata
    config={
    "architecture": "Vanilla CNN",
    "dataset": "UCMerced",
    "bs": BATCH_SIZE
    }
)

cb_wandb_aug = WandbMetricsLogger()

cb_aug = [cb_autosave_aug, cb_early_stop, cb_wandb_aug]

### **Training**

In [ ]:
history_aug = model_aug.fit(data_gen_train_aug,
                            epochs=EPOCHS,
                            validation_data=data_gen_val,
                            callbacks=cb_aug
                            )

wandb.finish()

Epoch 1/200
40/40 [==============================] - ETA: 0s - loss: 3.3278 - accuracy: 0.1437
Epoch 1: val_accuracy improved from -inf to 0.04762, saving model to cnn_aug.h5
40/40 [==============================] - 16s 299ms/step - loss: 3.3278 - accuracy: 0.1437 - val_loss: 6.7983 - val_accuracy: 0.0476
Epoch 2/200
40/40 [==============================] - ETA: 0s - loss: 2.7069 - accuracy: 0.1786
Epoch 2: val_accuracy did not improve from 0.04762
40/40 [==============================] - 10s 240ms/step - loss: 2.7069 - accuracy: 0.1786 - val_loss: 14.0544 - val_accuracy: 0.0476
Epoch 3/200
40/40 [==============================] - ETA: 0s - loss: 2.4849 - accuracy: 0.2405
Epoch 3: val_accuracy did not improve from 0.04762
40/40 [==============================] - 10s 253ms/step - loss: 2.4849 - accuracy: 0.2405 - val_loss: 10.4133 - val_accuracy: 0.0476
Epoch 4/200
40/40 [==============================] - ETA: 0s - loss: 2.2728 - accuracy: 0.3040
Epoch 4: val_accuracy did not improve fr

epoch/accuracy,▁▂▃▄▅▆▆▆▆▇▇▇▇▇██████████████████████████
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁▁▁▁▃▄▅▆▅▇▇▇▄▇▇▇▇▇▇▇▇▇▅▇▇▇█▇█▇█▇██████▇█
epoch/val_loss,▄▆▄█▃▂▂▁▂▁▁▁▃▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,0.94365
epoch/epoch,108
epoch/learning_rate,0.001
epoch/loss,0.17438
epoch/val_accuracy,0.8


## **Wandb logs**

Model training report: [Vanilla CNN for image classification using UCMerced](https://api.wandb.ai/links/igp-tubs/g9eamf3j)

## **Testing the model**

In [ ]:
# Run this line to load a trained model

# model_orig.load_weights("cnn_orig.h5")
# model_aug.load_weights("cnn_aug.h5")

In [ ]:
data_gen_test = DataGenerator(path_images=x_test,
                              labels=y_test,
                              batch_size=BATCH_SIZE,
                              n_classes=n_classes,
                              target_size=TARGET_SIZE,
                              shuffle=False)

print("Train:")
scores_train_orig = model_orig.evaluate(data_gen_train)
scores_train_aug = model_aug.evaluate(data_gen_train_aug)

print("Validation:")
scores_val_orig = model_orig.evaluate(data_gen_val)
scores_val_aug = model_aug.evaluate(data_gen_val)

print("Test:")
scores_test_orig = model_orig.evaluate(data_gen_test)
scores_test_aug = model_aug.evaluate(data_gen_test)

Train:
40/40 [==============================] - 8s 192ms/step - loss: 0.1525 - accuracy: 0.9532
Validation:
14/14 [==============================] - 2s 151ms/step - loss: 1.2700 - accuracy: 0.8000
Test:
14/14 [==============================] - 2s 153ms/step - loss: 1.4794 - accuracy: 0.7714


<center>

|             |Loss &darr;|              ||Accuracy &uarr;|      |
|:------------|:-------:|:------------:||:-------:|:------------:|
|**Model**    |**Train**|**Validation**||**Train**|**Validation**|
|w/o data augmentation| 0.96   |     4.64     ||   0.86  |     0.51     |
|w/ data augmentation |**0.15** |  **1.27**    || **0.95**|   **0.80**   |

<br/><br/>

|             |Loss &darr;||Accuracy &uarr;|
|:------------|:------:||:-------:|
|**Model**    |**Test**||**Test** |
|w/o data augmentation|  4.29  ||  0.50   |
|w/ data augmentation |**1.48**|| **0.77**|

</center>
